In [1]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
import warnings

In [2]:
train_data=pd.read_csv("https://raw.githubusercontent.com/BhavanishDhamnaskar/twitterdata/main/twitter_training.csv",names=["Tweet_ID","entity","sentiment","Tweet_content"])

In [3]:
train_data.head()

,Tweet_ID,entity,sentiment,Tweet_content
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [4]:
train_data.shape

(74682, 4)

In [5]:
test_data=pd.read_csv("https://raw.githubusercontent.com/BhavanishDhamnaskar/twitterdata/main/twitter_validation.csv",names=["Tweet_ID","entity","sentiment","Tweet_content"])


In [6]:
test_data.head()


,Tweet_ID,entity,sentiment,Tweet_content
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [7]:
test_data.shape


(1000, 4)

# **Let’s merge train and test dataset.**


In [8]:
df=pd.concat([train_data,test_data],axis=0)


In [9]:
df1=df.copy()

# **EDA**

In [10]:
df.shape

(75682, 4)

In [11]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 75682 entries, 0 to 999
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Tweet_ID       75682 non-null  int64 
 1   entity         75682 non-null  object
 2   sentiment      75682 non-null  object
 3   Tweet_content  74996 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.9+ MB


In [12]:
df.isnull().sum()


Tweet_ID           0
entity             0
sentiment          0
Tweet_content    686
dtype: int64

In [13]:
df.duplicated().sum()


3217

In [14]:
for i in df.columns:
  print(i,"has",df[i].nunique(),"unique values")


Tweet_ID has 12447 unique values
entity has 32 unique values
sentiment has 4 unique values
Tweet_content has 69974 unique values


In [15]:
df['entity'].unique()

array(['Borderlands', 'CallOfDutyBlackopsColdWar', 'Amazon', 'Overwatch',
       'Xbox(Xseries)', 'NBA2K', 'Dota2', 'PlayStation5(PS5)',
       'WorldOfCraft', 'CS-GO', 'Google', 'AssassinsCreed', 'ApexLegends',
       'LeagueOfLegends', 'Fortnite', 'Microsoft', 'Hearthstone',
       'Battlefield', 'PlayerUnknownsBattlegrounds(PUBG)', 'Verizon',
       'HomeDepot', 'FIFA', 'RedDeadRedemption(RDR)', 'CallOfDuty',
       'TomClancysRainbowSix', 'Facebook', 'GrandTheftAuto(GTA)',
       'MaddenNFL', 'johnson&johnson', 'Cyberpunk2077',
       'TomClancysGhostRecon', 'Nvidia'], dtype=object)

In [16]:
df['sentiment'].unique()


array(['Positive', 'Neutral', 'Negative', 'Irrelevant'], dtype=object)

In [17]:
df.head()

,Tweet_ID,entity,sentiment,Tweet_content
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [18]:
df.dtypes


Tweet_ID          int64
entity           object
sentiment        object
Tweet_content    object
dtype: object

All datatypes are correct and in accordance with data defination

# Dropping the null values

In [19]:
df = df.dropna(axis=0, how='any')


In [20]:
df.isnull().sum()


Tweet_ID         0
entity           0
sentiment        0
Tweet_content    0
dtype: int64

# Dropping the duplicate values

In [21]:
df.duplicated().sum()

2857

In [22]:
#Dropping the duplicates in the data.
df.drop_duplicates(inplace=True)


In [23]:
df.duplicated().sum()


0

In [24]:
df.describe()

,Tweet_ID
count,72139.000000
mean,6435.469593
std,3743.598918
min,1.000000
25%,3195.000000
50%,6432.000000
75%,9607.000000
max,13200.000000


# **TEXT PRE-PROCESSING**

 Lower Casing

In [25]:
 df['Tweet_content'].head()

0    im getting on borderlands and i will murder yo...
1    I am coming to the borders and I will kill you...
2    im getting on borderlands and i will kill you ...
3    im coming on borderlands and i will murder you...
4    im getting on borderlands 2 and i will murder ...
Name: Tweet_content, dtype: object

In [26]:
df["Tweet_content"] = df["Tweet_content"].str.lower()

In [27]:
df['Tweet_content'].head()


0    im getting on borderlands and i will murder yo...
1    i am coming to the borders and i will kill you...
2    im getting on borderlands and i will kill you ...
3    im coming on borderlands and i will murder you...
4    im getting on borderlands 2 and i will murder ...
Name: Tweet_content, dtype: object

# Removal of HTML Tags

In [28]:
## Remove html tags
import re
def remove_html_tags(text):
  pattern = re.compile('<.*?>')
  return pattern.sub(r'', text)

In [29]:
df['Tweet_content']=df['Tweet_content'].apply(remove_html_tags)


# Removal of URLs

In [30]:
##remove url
def remove_url(text):
  pattern = re.compile(r'https?://\S+|www\.\S+')
  return pattern.sub(r'', text)


In [31]:
df['Tweet_content']=df['Tweet_content'].apply(remove_url)


# Removal of Punctuations

In [32]:
## remove punctuations -
import string,time
string.punctuation
exclude = string.punctuation
def remove_punc(text):
  for char in exclude:
    text = text.replace(char,'')
  return text


In [33]:
df['Tweet_content']=df['Tweet_content'].apply(remove_punc)


# Chat Words Conversion

This is an important text preprocessing step if we are dealing with chat data. People do use a
lot of abbreviated words in chat and so it might be helpful to expand those words for our analysis
purposes.

In [34]:
 chat_words = {
"AFAIK": "As Far As I Know",
"AFK": "Away From Keyboard",
"ASAP": "As Soon As Possible",
"ATK": "At The Keyboard",
"ATM": "At The Moment",
"A3": "Anytime, Anywhere, Anyplace",
"BAK": "Back At Keyboard",
"BBL": "Be Back Later",
"BBS": "Be Back Soon",
"BFN": "Bye For Now",
"B4N": "Bye For Now",
"BRB": "Be Right Back",
"BRT": "Be Right There",
"BTW": "By The Way",
"B4": "Before",
"CU": "See You",
"CUL8R": "See You Later",
"CYA": "See You",
"FAQ": "Frequently Asked Questions",
"FC": "Fingers Crossed",
"FWIW": "For What It's Worth",
"FYI": "For Your Information",
"GAL": "Get A Life",
"GG": "Good Game",
"GN": "Good Night",
"GMTA": "Great Minds Think Alike",
"GR8": "Great!",
"G9": "Genius",
"IC": "I See",
"ICQ": "I Seek you (also a chat program)",
"ILU": "I Love You",
"IMHO": "In My Honest/Humble Opinion",
"IMO": "In My Opinion",
"IOW": "In Other Words",
"IRL": "In Real Life",
"KISS": "Keep It Simple, Stupid",
"LDR": "Long Distance Relationship",
"LMAO": "Laugh My A** Off",
"LOL": "Laughing Out Loud",
"LTNS": "Long Time No See",
"L8R": "Later",
"MTE": "My Thoughts Exactly",
"M8": "Mate",
"NRN": "No Reply Necessary",
"OIC": "Oh I See",
"PITA": "Pain In The A**",
"PRT": "Party",
"PRW": "Parents Are Watching",
"QPSA?": "Que Pasa?",
"ROFL": "Rolling On The Floor Laughing",
"ROFLOL": "Rolling On The Floor Laughing Out Loud",
"ROTFLMAO": "Rolling On The Floor Laughing My A** Off",
"SK8": "Skate",
"STATS": "Your sex and age",
"ASL": "Age, Sex, Location",
"THX": "Thank You",
"TTFN": "Ta-Ta For Now!",
"TTYL": "Talk To You Later",
"U": "You",
"U2": "You Too",
"U4E": "Yours For Ever",
"WB": "Welcome Back",
"WTF": "What The F***",
"WTG": "Way To Go!",
"WUF": "Where Are You From?",
"W8": "Wait...",
"7K": "Sick:-D Laughter",
"TFW": "That Feeling When",
"MFW": "My Face When",
"MRW": "My Reaction When",
"IFYP": "I Feel Your Pain",
"LOL": "Laughing Out Loud",
"TNTL": "Trying Not To Laugh",
"JK": "Just Kidding",
"IDC": "I Don't Care",
"ILY": "I Love You",
"IMU": "I Miss You",
"ADIH": "Another Day In Hell",
"ZZZ": "Sleeping, Bored, Tired",
"WYWH": "Wish You Were Here",
"TIME": "Tears In My Eyes",
"BAE": "Before Anyone Else",
"FIMH": "Forever In My Heart",
"BSAAW": "Big Smile And A Wink",
"BWL": "Bursting With Laughter",
"LMAO": "Laughing My A** Off",
"BFF": "Best Friends Forever",
"CSL": "Can't Stop Laughing"
}

In [35]:
def chat_conversion(text):
  new_text = []
  for w in text.split():
    if w.upper() in chat_words:
      new_text.append(chat_words[w.upper()])
    else:
      new_text.append(w)
  return " ".join(new_text)

In [36]:
 df['Tweet_content']=df['Tweet_content'].apply(chat_conversion)


# Removal of Emojis


In [37]:
import re
def remove_emoji(text):
  emoji_pattern = re.compile("["
                        u"\U0001F600-\U0001F64F" # emoticons
                        u"\U0001F300-\U0001F5FF" # symbols & pictographs
                        u"\U0001F680-\U0001F6FF" # transport & map symbols
                        u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        "]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', text)

In [38]:
df

,Tweet_ID,entity,sentiment,Tweet_content
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,i am coming to the borders and i will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you all
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
...,...,...,...,...
988,7516,LeagueOfLegends,Neutral,♥️ suikoden 2 1️⃣ alex kidd in miracle world 😢...
989,5708,HomeDepot,Positive,thank you to matching funds home depot rw paym...
991,2165,CallOfDuty,Neutral,late night stream with the boys come watch som...
995,4891,GrandTheftAuto(GTA),Irrelevant,⭐️ toronto is the arts and culture capital of ...


In [39]:
df['Tweet_content']=df['Tweet_content'].apply(remove_emoji)


In [40]:
df

,Tweet_ID,entity,sentiment,Tweet_content
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,i am coming to the borders and i will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you all
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
...,...,...,...,...
988,7516,LeagueOfLegends,Neutral,suikoden 2 1⃣ alex kidd in miracle world per...
989,5708,HomeDepot,Positive,thank you to matching funds home depot rw paym...
991,2165,CallOfDuty,Neutral,late night stream with the boys come watch som...
995,4891,GrandTheftAuto(GTA),Irrelevant,toronto is the arts and culture capital of ca...


# Spelling Correction





In [41]:
 from textblob import TextBlob

In [42]:
 def correct_spellings(text):
  textBlb = TextBlob(text)
  correct_text=textBlb.correct().string
  return correct_text

In [43]:
 !pip install pyspellchecker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 30.3 MB/s eta 0:00:00


In [44]:
 from spellchecker import SpellChecker

spell = SpellChecker()
def correct_spelling(text):
  corrected_text = []
  misspelled_words = spell.unknown(text.split())
  for word in text.split():
    if word in misspelled_words:
        corrected_text.append(spell.correction(word))
    else:
        corrected_text.append(word)
    return " ".join(corrected_text)
text = "speling correctin"
correct_spellings(text)

'spelling correction'

# Removal of stopwords


In [45]:
from nltk.corpus import stopwords


In [46]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [47]:
stopwords_list=stopwords.words('english')


In [49]:
def remove_stopwords(text):
  new_text = []

  for word in text.split():
    if word in stopwords.words('english'):
        new_text.append('')
    else:
        new_text.append(word)
  x = new_text[:]
  new_text.clear()
  return " ".join(x)

In [50]:
test_data['Tweet_content']=test_data['Tweet_content'].apply(remove_stopwords)

# Removal of Frequent words

In [52]:
from collections import Counter
cnt = Counter()
for text in test_data["Tweet_content"].values:
  for word in text.split():
    cnt[word] += 1

cnt.most_common(10)

[('I', 321),
 ('game', 79),
 ('like', 73),
 ('The', 69),
 ('get', 58),
 ('Johnson', 58),
 ('2', 57),
 ('&', 51),
 ('This', 46),
 ("I'm", 42)]

In [54]:
cnt.most_common(15)

[('I', 321),
 ('game', 79),
 ('like', 73),
 ('The', 69),
 ('get', 58),
 ('Johnson', 58),
 ('2', 57),
 ('&', 51),
 ('This', 46),
 ("I'm", 42),
 ('-', 41),
 ('love', 41),
 ('one', 41),
 ('play', 36),
 ('good', 33)]

In [55]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
  """custom function to remove the frequent words"""
  return " ".join([word for word in str(text).split() if word not in
  FREQWORDS])


In [57]:
df['Tweet_content'] = df['Tweet_content'].apply(lambda text:
  remove_freqwords(text))
df.head()

,Tweet_ID,entity,sentiment,Tweet_content
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,i am coming to the borders and i will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you all
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...


# Removal of Rare words

In [58]:
n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in
  RAREWORDS])

df['Tweet_content'] = df['Tweet_content'].apply(lambda text:
  remove_rarewords(text))
df.head()

,Tweet_ID,entity,sentiment,Tweet_content
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,i am coming to the borders and i will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you all
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...


# Removal of Emoticons

In [60]:
EMOTICONS = {
  u":‑\)":"Happy face or smiley",
  u":\)":"Happy face or smiley",
  u":-\]":"Happy face or smiley",
  u":\]":"Happy face or smiley",
  u":-3":"Happy face smiley",
  u":3":"Happy face smiley",
  u":->":"Happy face smiley",
  u":>":"Happy face smiley",
  u"8-\)":"Happy face smiley",
  u":o\)":"Happy face smiley",
  u":-\}":"Happy face smiley",
  u":\}":"Happy face smiley",
  u":-\)":"Happy face smiley",
  u":c\)":"Happy face smiley",
  u":\^\)":"Happy face smiley",
  u"=\]":"Happy face smiley",
  u"=\)":"Happy face smiley",
  u":‑D":"Laughing, big grin or laugh with glasses",
  u":D":"Laughing, big grin or laugh with glasses",
  u"8‑D":"Laughing, big grin or laugh with glasses",
  u"8D":"Laughing, big grin or laugh with glasses",
  u"X‑D":"Laughing, big grin or laugh with glasses",
  u"XD":"Laughing, big grin or laugh with glasses",
  u"=D":"Laughing, big grin or laugh with glasses",
  u"=3":"Laughing, big grin or laugh with glasses",
  u"B\^D":"Laughing, big grin or laugh with glasses",
  u":-\)\)":"Very happy",
  u":‑\(":"Frown, sad, andry or pouting",
  u":-\(":"Frown, sad, andry or pouting",
  u":\(":"Frown, sad, andry or pouting",
  u":‑c":"Frown, sad, andry or pouting",
  u":c":"Frown, sad, andry or pouting",
  u":‑<":"Frown, sad, andry or pouting",
  u":<":"Frown, sad, andry or pouting",
  u":‑\[":"Frown, sad, andry or pouting",
  u":\[":"Frown, sad, andry or pouting",
  u":-\|\|":"Frown, sad, andry or pouting",
  u">:\[":"Frown, sad, andry or pouting",
  u":\{":"Frown, sad, andry or pouting",
  u":@":"Frown, sad, andry or pouting",
  u">:\(":"Frown, sad, andry or pouting",
  u":'‑\(":"Crying",
  u":'\(":"Crying",
  u":'‑\)":"Tears of happiness",
  u":'\)":"Tears of happiness",
  u"D‑':":"Horror",
  u"D:<":"Disgust",
  u"D:":"Sadness",
  u"D8":"Great dismay",
  u"D;":"Great dismay",
  u"D=":"Great dismay",
  u"DX":"Great dismay",
  u":‑O":"Surprise",
  u":O":"Surprise",
  u":‑o":"Surprise",
  u":o":"Surprise",
  u":-0":"Shock",
  u"8‑0":"Yawn",
  u">:O":"Yawn",
  u":-\*":"Kiss",
  u":\*":"Kiss",
  u":X":"Kiss",
  u";‑\)":"Wink or smirk",
  u";\)":"Wink or smirk",
  u"\*-\)":"Wink or smirk",
  u"\*\)":"Wink or smirk",
  u";‑\]":"Wink or smirk",
  u";\]":"Wink or smirk",
  u";\^\)":"Wink or smirk",
  u":‑,":"Wink or smirk",
  u";D":"Wink or smirk",
  u":‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
  u":P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
  u"X‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
  u"XP":"Tongue sticking out, cheeky, playful or blowing a raspberry",
  u":‑Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
  u":Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
  u":b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
  u"d:":"Tongue sticking out, cheeky, playful or blowing a raspberry",
  u"=p":"Tongue sticking out, cheeky, playful or blowing a raspberry",
  u">:P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
  u":‑/":"Skeptical, annoyed, undecided, uneasy or hesitant",
  u":/":"Skeptical, annoyed, undecided, uneasy or hesitant",
  u":-[.]":"Skeptical, annoyed, undecided, uneasy or hesitant",
  u">:[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
  u">:/":"Skeptical, annoyed, undecided, uneasy or hesitant",
  u":[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
  u"=/":"Skeptical, annoyed, undecided, uneasy or hesitant",
  u"=[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
  u":L":"Skeptical, annoyed, undecided, uneasy or hesitant",
  u"=L":"Skeptical, annoyed, undecided, uneasy or hesitant",
  u":S":"Skeptical, annoyed, undecided, uneasy or hesitant",
  u":‑\|":"Straight face",
  u":\|":"Straight face",
  u":$":"Embarrassed or blushing",
  u":‑x":"Sealed lips or wearing braces or tongue-tied",
  u":x":"Sealed lips or wearing braces or tongue-tied",
  u":‑#":"Sealed lips or wearing braces or tongue-tied",
  u":#":"Sealed lips or wearing braces or tongue-tied",
  u":‑&":"Sealed lips or wearing braces or tongue-tied",
  u":&":"Sealed lips or wearing braces or tongue-tied",
  u"O:‑\)":"Angel, saint or innocent",
  u"O:\)":"Angel, saint or innocent",
  u"0:‑3":"Angel, saint or innocent",
  u"0:3":"Angel, saint or innocent",
  u"0:‑\)":"Angel, saint or innocent",
  u"0:\)":"Angel, saint or innocent",
  u":‑b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
  u"0;\^\)":"Angel, saint or innocent",
  u">:‑\)":"Evil or devilish",
  u">:\)":"Evil or devilish",
  u"\}:‑\)":"Evil or devilish",
  u"\}:\)":"Evil or devilish",
  u"3:‑\)":"Evil or devilish",
  u"3:\)":"Evil or devilish",
  u">;\)":"Evil or devilish",
  u"\|;‑\)":"Cool",
  u"\|‑O":"Bored",
  u":‑J":"Tongue-in-cheek",
  u"#‑\)":"Party all night",
  u"%‑\)":"Drunk or confused",
  u"%\)":"Drunk or confused",
  u":-###..":"Being sick",
  u":###..":"Being sick",
  u"<:‑\|":"Dump",
  u"\(>_<\)":"Troubled",
  u"\(>_<\)>":"Troubled",
  u"\(';'\)":"Baby",
  u"\(\^\^>``":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
  u"\(\^_\^;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
  u"\(-_-;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
  u"\(~_~;\) \(\.;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
  u"\(-_-\)zzz":"Sleeping",
  u"\(\^_-\)":"Wink",
  u"\(\(\+_\+\)\)":"Confused",
  u"\(\+o\+\)":"Confused",
  u"\(o\|o\)":"Ultraman",
  u"\^_\^":"Joyful",
  u"\(\^_\^\)/":"Joyful",
  u"\(\^O\^\)":"Joyful",
  u"\(\^o\^\)":"Joyful",
  u"\(__\)":"Kowtow as a sign of respect, or dogeza for apology",
  u"_\(\._\.\)_":"Kowtow as a sign of respect, or dogeza for apology",
  u"<\(_ _\)>":"Kowtow as a sign of respect, or dogeza for apology",
  u"<m\(__\)m>":"Kowtow as a sign of respect, or dogeza for apology",
  u"m\(__\)m":"Kowtow as a sign of respect, or dogeza for apology",
  u"m\(_ _\)m":"Kowtow as a sign of respect, or dogeza for apology",
  u"\('_'\)":"Sad or Crying",
  u"\(/_;\)":"Sad or Crying",
  u"\(T_T\) \(;_;\)":"Sad or Crying",
  u"\(;_;":"Sad of Crying",
  u"\(;_:\)":"Sad or Crying",
  u"\(;O;\)":"Sad or Crying",
  u"\(:_;\)":"Sad or Crying",
  u"\(ToT\)":"Sad or Crying",
  u";_;":"Sad or Crying",
  u";-;":"Sad or Crying",
  u";n;":"Sad or Crying",
  u";;":"Sad or Crying",
  u"Q\.Q":"Sad or Crying",
  u"T\.T":"Sad or Crying",
  u"QQ":"Sad or Crying",
  u"Q_Q":"Sad or Crying",
  u"\(-\.-\)":"Shame",
  u"\(-_-\)":"Shame",
  u"\(\)":"Shame",
  u"\(_\)":"Shame",
  u"\(=_=\)":"Tired",
  u"\(=\^\·\^=\)":"cat",
  u"\(=\^\·\·\^=\)":"cat",
  u"=_\^= ":"cat",
  u"\(\.\.\)":"Looking down",
  u"\(\._\.\)":"Looking down",
  u"\^m\^":"Giggling with hand covering mouth",
  u"\(\ \?":"Confusion",
  u"\(?_?\)":"Confusion",
  u">\^_\^<":"Normal Laugh",
  u"<\^!\^>":"Normal Laugh",
  u"\^/\^":"Normal Laugh",
  u"\ \*\^_\^\*" :"Normal Laugh",
  u"\(\^<\^\) \(\^\.\^\)":"Normal Laugh",
  u"\(^\^\)":"Normal Laugh",
  u"\(\^\.\^\)":"Normal Laugh",
  u"\(\^_\^\.\)":"Normal Laugh",
  u"\(\^_\^\)":"Normal Laugh",
  u"\(\^\^\)":"Normal Laugh",
  u"\(\^J\^\)":"Normal Laugh",
  u"\(\*\^\.\^\*\)":"Normal Laugh",
  u"\(\^—\^\ ":"Normal Laugh",
  u"\(#\^\.\^#\)":"Normal Laugh",
  u"\ \^—\^\ ":"Waving",
  u"\(;_;\)/~~~":"Waving",
  u"\(\^\.\^\)/~~~":"Waving",
  u"\(-_-\)/~~~ \($\·\·\)/~~~":"Waving",
  u"\(T_T\)/~~~":"Waving",
  u"\(ToT\)/~~~":"Waving",
  u"\(\*\^0\^\*\)":"Excited",
  u"\(\*_\*\)":"Amazed",
  u"\(\*_\*;":"Amazed",
  u"\(\+_\+\) \(@_@\)":"Amazed",
  u"\(\*\^\^\)v":"Laughing,Cheerful",
  u"\(\^_\^\)v":"Laughing,Cheerful",
  u"\(\(d[-_-]b\)\)":"Headphones,Listening to music",
  u'\(-"-\)':"Worried",
  u"\(;\)":"Worried",
  u"\(\^0_0\^\)":"Eyeglasses",
  u"\(\\ \)":"Happy",
  u"\(\\ \)":"Happy",
  u"\(\^\)o\(\^\)":"Happy",
  u"\(\^O\^\)":"Happy",
  u"\(\^o\^\)":"Happy",
  u"\)\^o\^\(":"Happy",
  u":O o_O":"Surprised",
  u"o_0":"Surprised",
  u"o\.O":"Surpised",
  u"\(o\.o\)":"Surprised",
  u"oO":"Surprised",
  u"\(\*m\)":"Dissatisfied",
  u"\(‘A`\)":"Snubbed or Deflated"
}

In [61]:
def remove_emoticons(text):
  emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
  return emoticon_pattern.sub(r'', text)

In [62]:
 df['Tweet_content']=df['Tweet_content'].apply(remove_emoticons)


# Conversion of Emoticon words

In [63]:
def convert_emoticons(text):
  for emot in EMOTICONS:
    text = re.sub(u'('+emot+')', "_".join(EMOTICONS[emot].replace(",","").
  split()), text)
    return text


# Stemming

In [64]:
from nltk.stem.porter import PorterStemmer

In [65]:
ps = PorterStemmer()
def stem_words(text):
  return " ".join([ps.stem(word) for word in text.split()])


In [66]:
df['Tweet_content']=df['Tweet_content'].apply(stem_words)

In [67]:
# after Stemming
df['Tweet_content'].head()

0       im get on borderland and i will murder you all
1      i am come to the border and i will kill you all
2         im get on borderland and i will kill you all
3      im come on borderland and i will murder you all
4    im get on borderland and i will murder you me all
Name: Tweet_content, dtype: object

In [68]:
# Before stemming
df1["Tweet_content"].head()


0    im getting on borderlands and i will murder yo...
1    I am coming to the borders and I will kill you...
2    im getting on borderlands and i will kill you ...
3    im coming on borderlands and i will murder you...
4    im getting on borderlands 2 and i will murder ...
Name: Tweet_content, dtype: object

Lemmatization

In [70]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [71]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])
df["text_lemmatized"] = df["Tweet_content"].apply(lambda text:
  lemmatize_words(text))
df.head()


,Tweet_ID,entity,sentiment,Tweet_content,text_lemmatized
0,2401,Borderlands,Positive,im get on borderland and i will murder you all,im get on borderland and i will murder you all
1,2401,Borderlands,Positive,i am come to the border and i will kill you all,i am come to the border and i will kill you all
2,2401,Borderlands,Positive,im get on borderland and i will kill you all,im get on borderland and i will kill you all
3,2401,Borderlands,Positive,im come on borderland and i will murder you all,im come on borderland and i will murder you all
4,2401,Borderlands,Positive,im get on borderland and i will murder you me all,im get on borderland and i will murder you me all


# Tokenization

In [72]:
from nltk.tokenize import word_tokenize,sent_tokenize


In [73]:
sent1 = 'I am going to visit delhi!'
word_tokenize(sent1)


['I', 'am', 'going', 'to', 'visit', 'delhi', '!']

In [74]:
df["text_tokenized"] = df["Tweet_content"].apply(lambda text:
  word_tokenize(text))


In [75]:
df["text_tokenized"].head()

0    [im, get, on, borderland, and, i, will, murder...
1    [i, am, come, to, the, border, and, i, will, k...
2    [im, get, on, borderland, and, i, will, kill, ...
3    [im, come, on, borderland, and, i, will, murde...
4    [im, get, on, borderland, and, i, will, murder...
Name: text_tokenized, dtype: object